In [22]:
import os

import ipywidgets as widgets


class FileBrowser(object):
    def __init__(self):
        self.path = os.getcwd()
        self._update_files()
        
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = self.path + "/" + f
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)
        
    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box
    
    def _update(self, box):
        
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
            else:
                self.path = self.path + "/" + b.description
            self._update_files()
            self._update(box)
        
        buttons = []
        if self.files:
            button = widgets.Button(description='..', background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.dirs:
            button = widgets.Button(description=f, background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.files:
            button = widgets.Button(description=f)
            button.on_click(on_click)
            buttons.append(button)
        box.children = tuple([widgets.HTML("<h2>%s</h2>" % (self.path,))] + buttons)

# example usage:
f = FileBrowser()
f.widget()

VBox(children=(HTML(value='<h2>d:\\projecten\\dotter\\dotterdev\\dotter\\examples\\notebooks</h2>'), Button(description='..', style=ButtonStyle()), Button(description='.ipynb_checkpoints', style=ButtonStyle()), Button(description='images', style=ButtonStyle()), Button(description='dotter.log', style=ButtonStyle()), Button(description='frictionvalues.csv', style=ButtonStyle()), Button(description='ParameterEstimation.ipynb', style=ButtonStyle()), Button(description='Untitled.ipynb', style=ButtonStyle()), Button(description='VirtualRiver.ipynb', style=ButtonStyle())))

In [23]:

#   <interact with widget, select a path>
# in a separate cell:
f.path # returns the selected path


'd:\\projecten\\dotter\\dotterdev\\dotter\\examples\\notebooks'

In [24]:


import base64

try:
    from io import StringIO
except ImportError:
    from StringIO import StringIO

import pandas

from traitlets import Unicode, Instance, observe
from ipywidgets import DOMWidget
from IPython import display



In [25]:
%%html
<style>
    .uploader-widget{
        border: dashed 2px grey;
        opacity: 0.5;
        background-color: #efefef;
        color: grey;
        position: relative;
    }
    .uploader-widget.uploader-dragged {
        background-color: #333;
        color: white;
    }
    .uploader-widget input{
        opacity: 0;
        width: 100%;
        position: absolute;
        width: 100%;
        height: 100%;
        top: 0;
    }
    .uploader-widget h3 {
        text-align: center;
        margin: 0;
        padding: 20px;
    }
</style>

In [26]:


%%javascript
require.undef("widget-uploader")
define(
"widget-uploader",
[
    "underscore",
    "jquery",
    "nbextensions/widgets/widgets/js/widget"
],
function(_, $, widget){
    var UploaderView = widget.DOMWidgetView.extend({
        className: "uploader-widget",
        events: {
            dragenter: "onDragEnter",
            dragleave: "onDragLeave",
            drop: "onDrop",
            "change input": "onChangeInput"
        },
        
        render: function(){
            var that = this;

            this.$label = $("<h3/>").appendTo(this.$el);

            this.$fileField = $("<input/>", {type: "file"})
                .appendTo(this.$el);

            this.update();
        },
        onChangeInput: function(){
            this.setFile(this.$fileField[0].files[0]);
        },
        setFile: function(file){
            var reader  = new FileReader();

            reader.addEventListener("load", _.bind(function(){
                this.model.set("base64_data", reader.result);
                this.touch();
            }, this), false);

            reader.readAsDataURL(file);
        },
        update: function() {
            this.$label.text(this.model.get("label"));
        },
                                                   
        onDragEnter: function(evt){ this.$el.addClass("uploader-dragged"); console.log(evt);},
        onDragLeave: function(evt){ this.$el.removeClass("uploader-dragged"); console.log(evt);},
        onDrop: function(evt){
            evt.preventDefault();
            evt.stopImmediatePropagation();
            this.$el.removeClass("uploader-dragged");
            console.log(evt);
            this.setFile(evt.originalEvent.dataTransfer.files[0]);
        }
    });
    return {
        UploaderView: UploaderView
    }
});



<IPython.core.display.Javascript object>

In [27]:


class BaseUploaderWidget(DOMWidget):
    _view_module = Unicode("widget-uploader").tag(sync=True)
    _view_name = Unicode("UploaderView").tag(sync=True)
    base64_data = Unicode().tag(sync=True)
    label = Unicode("Upload a File").tag(sync=True)
    
    @observe("base64_data")
    def _update_label(self, change):
        self.label = self._label() or self.label
    
    def _label(self):
        return "{} bytes uploaded {}".format(
            len(self.base64_data),
            self.base64_data.split(";")[0])

uploader = BaseUploaderWidget()
uploader
print (uploader)


BaseUploaderWidget()


In [28]:
class CSVUploader(BaseUploaderWidget):
    dataframe = Instance(klass=pandas.DataFrame, allow_none=True)
    label = Unicode("Upload a CSV File").tag(sync=True)
    
    def _label(self):
        self.label = "{} rows by {} columns".format(
            len(self.dataframe),
            len(self.dataframe.columns),
        )

    @observe("base64_data")
    def _update_dataframe(self, new):
        csv = StringIO()
        csv.write(u"{}".format(base64.decodestring(self.base64_data.split(",")[1])))
        csv.seek(0)
        self.dataframe = pandas.read_csv(csv)

In [29]:
csv = CSVUploader()
csv

CSVUploader()

In [30]:


csv.dataframe

